# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 9:42AM,241746,22,620692,"NBTY Global, Inc.",Released
1,Jul 13 2022 9:40AM,241745,12,U0963,"Uniderm USA, Inc",Released
2,Jul 13 2022 9:36AM,241744,12,HH6314,Hush Hush,Released
3,Jul 13 2022 9:36AM,241740,19,8269444,Else Nutrition,Released
4,Jul 13 2022 9:35AM,241742,12,HH-35817,Hush Hush,Released
5,Jul 13 2022 9:35AM,241742,12,HH-35818,Hush Hush,Released
6,Jul 13 2022 9:35AM,241742,12,HH-35819,Hush Hush,Released
7,Jul 13 2022 9:35AM,241742,12,HH-35820,Hush Hush,Released
8,Jul 13 2022 9:35AM,241742,12,HH-35821,Hush Hush,Released
9,Jul 13 2022 9:35AM,241742,12,HH-35822,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,241742,Released,28
51,241743,Released,1
52,241744,Released,1
53,241745,Released,1
54,241746,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241742,NaN,NaN,28.0
241743,NaN,NaN,1.0
241744,NaN,NaN,1.0
241745,NaN,NaN,1.0
241746,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0.0,0.0,22.0
241635,0.0,5.0,6.0
241644,0.0,0.0,1.0
241648,0.0,1.0,4.0
241654,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0,0,22
241635,0,5,6
241644,0,0,1
241648,0,1,4
241654,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,0,0,22
1,241635,0,5,6
2,241644,0,0,1
3,241648,0,1,4
4,241654,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,,,22
1,241635,,5,6
2,241644,,,1
3,241648,,1,4
4,241654,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc."
1,Jul 13 2022 9:40AM,241745,12,"Uniderm USA, Inc"
2,Jul 13 2022 9:36AM,241744,12,Hush Hush
3,Jul 13 2022 9:36AM,241740,19,Else Nutrition
4,Jul 13 2022 9:35AM,241742,12,Hush Hush
32,Jul 13 2022 9:34AM,241743,12,Hush Hush
33,Jul 13 2022 9:33AM,241741,20,Alumier Labs Inc.
34,Jul 13 2022 9:23AM,241739,10,"Citieffe, Inc."
35,Jul 13 2022 9:22AM,241738,20,Alumier Labs Inc.
52,Jul 13 2022 9:16AM,241737,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",,,1
1,Jul 13 2022 9:40AM,241745,12,"Uniderm USA, Inc",,,1
2,Jul 13 2022 9:36AM,241744,12,Hush Hush,,,1
3,Jul 13 2022 9:36AM,241740,19,Else Nutrition,,,1
4,Jul 13 2022 9:35AM,241742,12,Hush Hush,,,28
5,Jul 13 2022 9:34AM,241743,12,Hush Hush,,,1
6,Jul 13 2022 9:33AM,241741,20,Alumier Labs Inc.,,,1
7,Jul 13 2022 9:23AM,241739,10,"Citieffe, Inc.",,,1
8,Jul 13 2022 9:22AM,241738,20,Alumier Labs Inc.,,,17
9,Jul 13 2022 9:16AM,241737,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",1,,
1,Jul 13 2022 9:40AM,241745,12,"Uniderm USA, Inc",1,,
2,Jul 13 2022 9:36AM,241744,12,Hush Hush,1,,
3,Jul 13 2022 9:36AM,241740,19,Else Nutrition,1,,
4,Jul 13 2022 9:35AM,241742,12,Hush Hush,28,,
5,Jul 13 2022 9:34AM,241743,12,Hush Hush,1,,
6,Jul 13 2022 9:33AM,241741,20,Alumier Labs Inc.,1,,
7,Jul 13 2022 9:23AM,241739,10,"Citieffe, Inc.",1,,
8,Jul 13 2022 9:22AM,241738,20,Alumier Labs Inc.,17,,
9,Jul 13 2022 9:16AM,241737,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",1,,
1,Jul 13 2022 9:40AM,241745,12,"Uniderm USA, Inc",1,,
2,Jul 13 2022 9:36AM,241744,12,Hush Hush,1,,
3,Jul 13 2022 9:36AM,241740,19,Else Nutrition,1,,
4,Jul 13 2022 9:35AM,241742,12,Hush Hush,28,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 13 2022 9:40AM,241745,12,"Uniderm USA, Inc",1.0,NaN,NaN
2,Jul 13 2022 9:36AM,241744,12,Hush Hush,1.0,NaN,NaN
3,Jul 13 2022 9:36AM,241740,19,Else Nutrition,1.0,NaN,NaN
4,Jul 13 2022 9:35AM,241742,12,Hush Hush,28.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:42AM,241746,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 13 2022 9:40AM,241745,12,"Uniderm USA, Inc",1.0,0.0,0.0
2,Jul 13 2022 9:36AM,241744,12,Hush Hush,1.0,0.0,0.0
3,Jul 13 2022 9:36AM,241740,19,Else Nutrition,1.0,0.0,0.0
4,Jul 13 2022 9:35AM,241742,12,Hush Hush,28.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5800823,187.0,67.0,7.0
12,966974,31.0,0.0,0.0
15,483313,42.0,0.0,0.0
16,725172,21.0,2.0,0.0
18,241669,0.0,1.0,0.0
19,1208540,25.0,10.0,0.0
20,966830,53.0,3.0,0.0
21,966759,4.0,0.0,0.0
22,241746,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5800823,187.0,67.0,7.0
1,12,966974,31.0,0.0,0.0
2,15,483313,42.0,0.0,0.0
3,16,725172,21.0,2.0,0.0
4,18,241669,0.0,1.0,0.0
5,19,1208540,25.0,10.0,0.0
6,20,966830,53.0,3.0,0.0
7,21,966759,4.0,0.0,0.0
8,22,241746,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,187.0,67.0,7.0
1,12,31.0,0.0,0.0
2,15,42.0,0.0,0.0
3,16,21.0,2.0,0.0
4,18,0.0,1.0,0.0
5,19,25.0,10.0,0.0
6,20,53.0,3.0,0.0
7,21,4.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,187.0
1,12,Released,31.0
2,15,Released,42.0
3,16,Released,21.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,67.0,0.0,0.0,2.0,1.0,10.0,3.0,0.0,0.0
Released,187.0,31.0,42.0,21.0,0.0,25.0,53.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,67.0,0.0,0.0,2.0,1.0,10.0,3.0,0.0,0.0
2,Released,187.0,31.0,42.0,21.0,0.0,25.0,53.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,67.0,0.0,0.0,2.0,1.0,10.0,3.0,0.0,0.0
2,Released,187.0,31.0,42.0,21.0,0.0,25.0,53.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()